In [1]:
import os
os.chdir("../../../../")
from scripts.python.PdfParse import *
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

In [2]:
tonga_lsts = os.listdir(os.getcwd() + "/data/tourism/tonga/scraping/")
filepaths = []
for path in tonga_lsts:
    folder_path = os.getcwd() + "/data/tourism/tonga/scraping/"
    if "Dec" in path:
        filepaths.append(folder_path + path)
    elif "2021" and "Bulletin" in path:
        filepaths.append(folder_path + path)
    else:
        pass

In [3]:
months = pd.DataFrame()

for file in filepaths[:-1]:
    print(file)
    
    fileyear = file.split("/")[-1].split("-")[-1].replace(".pdf", "")
    
    df = load_pdf(file, "Monthly Arrival and Departure", table_page=-5)
    latest_year, year_idx, month_idx = split_time(df, "Period")
    month = df.iloc[month_idx, 0:4]
    start_year = detect_year(df.iloc[month_idx].iloc[0])

    month = (month.dropna(how="all").reset_index()
             .drop("index", axis=1))

    print(f"The file starts from {start_year}.")

    month = separate_data(month, "Air Ship", " ").drop("Air Ship", axis=1)
    month = remove_separator(month)
    month = month.replace(r'^\s*$', 0, regex=True)

    if check_quality(month, ["Period", "Year"], "Total") == False:
        name = file.split("/")[-1].split(".")[0]
        print("  ", name, "could go wrong!")

    generate_time(month, start_year)
    month["fileyear"] = fileyear
    months = pd.concat([months, month], axis=0)
    
    
months = (months.drop_duplicates()
          .sort_values(by=["Year"])
          .reset_index()
          .drop("index", axis=1))

/Users/czhang/Desktop/pacific-observatory/data/tourism/tonga/scraping/Statistical-Bulletin-on-International-Arrivals-and-Departures-2021.pdf
The file starts from 2018.
/Users/czhang/Desktop/pacific-observatory/data/tourism/tonga/scraping/Migration-Report-Dec-2017.pdf
The file starts from 2013.
/Users/czhang/Desktop/pacific-observatory/data/tourism/tonga/scraping/Migration-December-Report-2019.pdf
The file starts from 2018.
/Users/czhang/Desktop/pacific-observatory/data/tourism/tonga/scraping/12-December-Migration-Report-2014.pdf


Got stderr: Jun 08, 2023 7:00:15 PM org.apache.pdfbox.pdmodel.PDDocument importPage
Jun 08, 2023 7:00:15 PM org.apache.pdfbox.pdmodel.PDDocument importPage



The file starts from 2010.
/Users/czhang/Desktop/pacific-observatory/data/tourism/tonga/scraping/12-December-Migration-Report-2015.pdf


Got stderr: Jun 08, 2023 7:00:23 PM org.apache.pdfbox.pdmodel.PDDocument importPage
Jun 08, 2023 7:00:23 PM org.apache.pdfbox.pdmodel.PDDocument importPage



The file starts from 2010.
/Users/czhang/Desktop/pacific-observatory/data/tourism/tonga/scraping/Migration-December-Report-2020.pdf
The file starts from 2018.
/Users/czhang/Desktop/pacific-observatory/data/tourism/tonga/scraping/12-December-Migration-Report-2013.pdf


Got stderr: Jun 08, 2023 7:00:33 PM org.apache.pdfbox.pdmodel.PDDocument importPage
Jun 08, 2023 7:00:33 PM org.apache.pdfbox.pdmodel.PDDocument importPage



The file starts from 2010.
/Users/czhang/Desktop/pacific-observatory/data/tourism/tonga/scraping/12-Migration-Report-Dec-2016.pdf


Got stderr: Jun 08, 2023 7:00:41 PM org.apache.pdfbox.pdmodel.PDDocument importPage
Jun 08, 2023 7:00:41 PM org.apache.pdfbox.pdmodel.PDDocument importPage



The file starts from 2011.


In [4]:
# Clean the datetime format
dates = list()
for idx in months.index:
    month, year = months["Period"][idx], str(months["Year"][idx])
    if type(month) == str:
        try:
            YM = year + month
            dates.append(datetime.strptime(YM, "%Y%B"))
        except:
            dates.append(datetime.strptime(YM, "%Y%b"))
    else:
        dates.append(month)

months["date"] = dates
months = months.sort_values(by=["date", "fileyear"])

In [5]:
def check_combination(data, incl_cols, target, error_idx):
    
    check_df = data.loc[error_idx, incl_cols]
    for idx, row in check_df.iterrows():
        init_str = ''
        for row_idx in range(len(row)):
            init_str = init_str + str(row[row_idx])
        if str(data[target][idx]) == init_str:
            error_idx.remove(idx)
            check_df.loc[idx][0] = float(init_str)
            check_df.loc[idx][-1] = float(0)       
    return check_df, error_idx

In [6]:
error_idx = check_quality(months, ["Period", "Year", "date", "fileyear"], "Total")
corrected_df, error_idx = check_combination(months, ["Air", "Ship"], "Total", error_idx)
months.loc[corrected_df.index, ["Air", "Ship"]] = corrected_df

In [7]:
months = (months[~months.date.isna()].fillna(0)
          .sort_values(by=["date", "fileyear"])
          .reset_index().drop(["Period", "index"], axis=1))

for col in ["Total", "Air", "Ship", "Yacht"]:
    months[col] = months[col].astype(int)

In [8]:
temp = months[months.columns[~months.columns.isin(["fileyear"])]]
nodup_idx = temp.drop_duplicates().index
months = (months.iloc[nodup_idx]
          .groupby("date")
          .last()
          .reset_index()
          .drop("fileyear", axis=1)
          .rename({"Total": "All", 
                   "Air": "Total"}, axis=1))

In [10]:
# Check for duplicates (e.g. Dec vs December in different years)

# indexes = months[colnames].drop_duplicates().index
# months = months.iloc[indexes].reset_index().drop("index", axis=1)

#Save the file
months.to_csv(os.getcwd() + "/data/tourism/tonga/tonga_monthly_visitor.csv",
              encoding="utf-8")